In [172]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from numpy import argsort
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.evaluation import *
from Functions.NLP.alertp1_nlp import *
from Functions.NLP.data_with_nlp import *
from Functions.pipeline import *
from gensim.models import Word2Vec
import re
import nltk
import gensim

In [173]:
creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
1123


In [174]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)

In [175]:
#data= pre_process(alertP1)

In [176]:
data.columns

Index(['ID_DOENTE', 'PROCESSO', 'COD_REFERENCIA', 'COD_PZ',
       'COD_UNID_SAUDE_PROV', 'UNID_PROV', 'TIPO_UNID', 'COD_CTH_PRIOR',
       'CTH_PRIOR', 'COD_MOTIVO_RECUSA', 'DES_MOTIVO_RECUSA',
       'COD_ESPECIALIDADE', 'DES_ESPECIALIDADE', 'agrupadora',
       'OUTRA_ENTIDADE', 'DATA_RECEPCAO', 'DATA_ENVIO', 'DATA_RETORNO',
       'NUM_TAXA', 'ESTADO', 'DATA_MARCACAO', 'DATA_REALIZACAO', 'OBSERVACOES',
       'Mês_entrada', 'Ano_entrada', 'trata data recusa', 'resume saída',
       'mês_saida', 'ano_saida', 'Texto', 'clean_text', 'result',
       'before_accepted', 'text_length', 'symptom_1', 'symptom_0',
       'exam_identified', 'comorbidity_identified', 'medication_level_1',
       'medication_level_2', 'medication_level_3', 'medication_count',
       'medication_concentration', 'count_Alteração', 'count_Agravamento',
       'count_HGO', 'count_Estável', 'outside area', 'SAM', 'SON', 'unknown',
       'Other specialities', '2', '3+', 'HOSP', 'UCSP', 'USF A', 'USF B',
       'out

In [143]:
import math

# Split data into train and test
AlertP1_sorted = data[data['Texto']!=''].sort_values(by='DATA_RECEPCAO')



# split the data frame into test and train sets
train_set = AlertP1_sorted
# Import librariers 
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings(action="ignore")
# Get rid of special characters and transform Texto column to Latin words
train_set['Texto'] = train_set['Texto'].apply(lambda x: unidecode(x))

#The re.sub function is used to substitute all digits (\d) with an empty string
train_set['Texto'] = train_set['Texto'].apply(lambda x: re.sub(r'\d', '', x))

# Remove all names in Texto variable
# This function uses a regular expression to find all words in the text that start with a 
# capital letter (\b[A-Z][a-z]+\b), which are assumed to be names
text = train_set['Texto'] 

# remove all hyphens from the text
text = text.replace('-', '')

def remove_names(text):
    # Find all words that start with a capital letter
    names = re.findall(r'\b[A-Z][a-z]+\b', text)
    
    # Replace the names with an empty string
    for name in names:
        text = text.replace(name, '')
        
    return text
# Create an empty list to store the text
text_list = []

# Loop through the 'text' column
for text in text.str.lower(): # Transform every word to lower case
    text_list.append(text)

# Print the list of text
#print(text_list)
# Download the Portuguese stop words
nltk.download('stopwords')
nltk.download('punkt')

# Get the Portuguese stop words
stop_words = set(stopwords.words('portuguese'))

# Manually remove stopwords
stop_words.update(['-//','.', ',','(',')',':','-','?','+','/',';','2','1','drª','``','','3','desde','anos','doente','consulta','alterações','se',"''",'cerca','refere','hgo','utente','vossa','s','...','ainda','c','filha','costa','dr.','pereira','ja','--','p','dr','h','n','>','q','//','..','b','++','%','//','-','+++/','=','+++/'])

# Create a new list to store the filtered text
filtered_text = []

# Loop through the text_list and remove the stop words
for text in text_list:
    words = word_tokenize(text)
    words = [word for word in words if word.lower() not in stop_words]
    filtered_text.append(" ".join(words))

# Print the filtered text
#print(filtered_text)
# Save the filtered text as a new column to the dataframe
train_set['filtered_text'] = filtered_text
train_set.columns

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/utkuserdaroglu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/utkuserdaroglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/utkuserdaroglu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/utkuserdaroglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Index(['ID_DOENTE', 'PROCESSO', 'COD_REFERENCIA', 'COD_PZ',
       'COD_UNID_SAUDE_PROV', 'UNID_PROV', 'TIPO_UNID', 'COD_CTH_PRIOR',
       'CTH_PRIOR', 'COD_MOTIVO_RECUSA', 'DES_MOTIVO_RECUSA',
       'COD_ESPECIALIDADE', 'DES_ESPECIALIDADE', 'agrupadora',
       'OUTRA_ENTIDADE', 'DATA_RECEPCAO', 'DATA_ENVIO', 'DATA_RETORNO',
       'NUM_TAXA', 'ESTADO', 'DATA_MARCACAO', 'DATA_REALIZACAO', 'OBSERVACOES',
       'Mês_entrada', 'Ano_entrada', 'trata data recusa', 'resume saída',
       'mês_saida', 'ano_saida', 'Texto', 'clean_text', 'result',
       'before_accepted', 'text_length', 'symptom_1', 'symptom_0',
       'exam_identified', 'comorbidity_identified', 'medication_level_1',
       'medication_level_2', 'medication_level_3', 'medication_count',
       'medication_concentration', 'count_Alteração', 'count_Agravamento',
       'count_HGO', 'count_Estável', 'outside area', 'SAM', 'SON', 'unknown',
       'Other specialities', '2', '3+', 'HOSP', 'UCSP', 'USF A', 'USF B',
       'out

In [144]:
# Define function for lemmatization
def spacy_lemmatizer(df):
    import spacy
    import pt_core_news_md
    nlp = pt_core_news_md.load()

    doclist = list(nlp.pipe(df))

    docs=[]
    for i, doc in enumerate(doclist):
        docs.append(' '.join([listitem.lemma_ for listitem in doc]))
        
    return docs

In [145]:
# create an empty list to store the words
word_list = []

# loop through each row of the "text_column" column
for index, row in train_set.iterrows():
    
    # split the text into individual words using whitespace as a delimiter
    words = row['filtered_text'].split()
    # add the words to the word list
    word_list.extend(words)

# print the word list
print(word_list)

['idade', 'tremor', 'acentuado', 'membros', 'superiores', 'agravamento', 'ultimo', 'ano', 'interferindo', 'realizacao', 'actividades', 'vida', 'diaria', 'conducao', 'filho', 'esquizofrenia', 'nao', 'consegue', 'levar', 'instituicao', 'onde', 'passava', 'dia', 'dificuldades', 'conducao', 'relatorio', 'clinico', 'traz', 'inumeros', 'relatorios', 'idas', 'hgo-su', 'vulvo-vaginites', 'durante', 'gravidez', 'relatorios', 'sobre', 'ida', 'crise', 'epilepsia', 'tendo', 'sido', 'medicada', 'depakine', 'chrono', 'enquanto', 'nao', 'completasse', 'todo', 'esclarecimento', 'situacao', 'ter', 'tido', 'crises', 'epilepticas', 'durante', 'gravidez', 'marcada', 'cons', 'neurologia', 'julho/', 'diz', 'nao', 'recebeu', 'carta', 'casa', 'tremor', 'fino', 'ambas', 'maos', 'boca', 'ha', 'meses', 'deixa', 'cair', 'objectos', 'nao', 'consegue', 'escrever', 'peco', 'cons', 'dc', 'movimento', 'pe', 'pendente', 'instalacao', 'ha', 'meses', 'atrofia', 'muscular', 'neuropatia', 'periferica', 'mononeuropatia', 'h

In [146]:
Lemma = spacy_lemmatizer(word_list) # Call lemmatizer function

# print length of word_list and compare the count after doing lemmatization
from collections import Counter

items = Counter(Lemma).keys()
print('The number of words after lemmatization:',len(items))

items2 = Counter(word_list).keys()
print('The number of words before lemmatization:',len(items2))

The number of words after lemmatization: 8948
The number of words before lemmatization: 11027


In [160]:
# apply the spacy_lemmatizer function to each row in the 'text' column
train_set['text_lemmatized'] = spacy_lemmatizer(train_set['filtered_text'])

# drop rows with empty strings
train_set_filtered = train_set[['text_lemmatized','filtered_text','result']].replace('', pd.NA).dropna()
train_set_filtered = pd.DataFrame(train_set_filtered)
train_set_filtered

,text_lemmatized,filtered_text,result
630,idade tremor acentuar membro alto agravamento ...,idade tremor acentuado membros superiores agra...,1
1537,relatorio clinico,relatorio clinico,0
985,trazer inumero relatorio ida hgo-su vulvo-vagi...,traz inumeros relatorios idas hgo-su vulvo-vag...,1
1103,tremor fino ambos mao bocar ha mês deixar cair...,tremor fino ambas maos boca ha meses deixa cai...,1
752,pe pendente instalacao ha mês atrofiar muscula...,pe pendente instalacao ha meses atrofia muscul...,1
...,...,...,...
1606,disfagia solir ser consultar cons gastroentero...,disfagia solidos sido consultado cons gastroen...,1
1335,sra seguir psiquiatria doenca bipolar longo da...,sra seguida psiquiatria doenca bipolar longa d...,1
1177,sequela parto anoxia cerebral seguir inicialme...,sequelas parto anoxia cerebral seguido inicial...,1
1332,episodio frequente desorientacao espacial oper...,episodios frequentes desorientacao espacial op...,0


In [161]:
# Create a lambda function to apply to the DataFrame. I didint get the code below. 
#train_set_filtered['accepted/rejected'] = train_set_filtered['result'].apply(lambda x: 1 if x == 'Accepted' else 0)
#train_set_filtered.to_clipboard()


In [162]:
# Import Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Create a TF-IDF vectorizer object with max_df = 0.8 and min_df = 5
tfidf_vectorizer = TfidfVectorizer() #I removed max_df= 0.8

# Fit and transform the training data into a sparse matrix of TF-IDF features
X_train_tfidf = tfidf_vectorizer.fit_transform(train_set_filtered['text_lemmatized'])

# Get the vocabulary
vocab = tfidf_vectorizer.get_feature_names()
print(vocab)


print(type(tfidf_vectorizer))

print(type(X_train_tfidf))
print(type(vocab))
print(tfidf_vectorizer)













# Transform the test data into a sparse matrix of TF-IDF features
#X_test_tfidf = tfidf_vectorizer.transform(train_set_filtered['filtered_text'])

# Use the TF scores for future predictions
#tf_vectorizer = TfidfVectorizer(use_idf=False, max_df=0.8, min_df=5)
#X_tf = tf_vectorizer.fit_transform(train_set_filtered['filtered_text'])

# Use the TF-IDF scores for logistic regression with other variables
#X_train_tfidf_lr = pd.concat([pd.DataFrame(X_train_tfidf.toarray()), train_set_filtered[train_set_filtered.index.isin(X_train.index)].drop(['filtered_text', 'accepted/rejected'], axis=1)], axis=1)
#X_test_tfidf_lr = pd.concat([pd.DataFrame(X_test_tfidf.toarray()), train_set_filtered[train_set_filtered.index.isin(X_test.index)].drop(['filtered_text', 'accepted/rejected'], axis=1)], axis=1)

['2e', 'aa', 'aacentuar', 'aas', 'aat', 'ab', 'abaixo', 'abandonar', 'abandono', 'abcesso', 'abd', 'abdomen', 'abdominal', 'abducaor', 'abducente', 'abdutor', 'abenular', 'aberto', 'abertura', 'aberturar', 'abolido', 'abolir', 'abordagem', 'abotoar', 'abr', 'abril', 'abrir', 'abstinecer', 'abstinencia', 'abstinente', 'abstracaor', 'abulico', 'abundante', 'abuso', 'abvaliacao', 'abx', 'ac', 'acabar', 'acamada', 'acamar', 'acarbose', 'acaro', 'acatisia', 'accaar', 'accao', 'acd', 'acdr', 'ace', 'aceder', 'aceitar', 'acelarar', 'aceleracao', 'acelerar', 'acenrtuar', 'acentuacao', 'acentuacaor', 'acentuacoa', 'acentuadas', 'acentuado', 'acentuar', 'acer', 'acerca', 'acerto', 'acesor', 'acessivel', 'acesso', 'acetilsalicilico', 'acetilssalicilico', 'acetona', 'acfol', 'achado', 'achao', 'achar', 'achiv', 'aci', 'aciclovir', 'acidente', 'acido', 'acidose', 'acima', 'acinesia', 'acinetico', 'aclargamento', 'aclidinio', 'acm', 'aco', 'acoluna', 'acomapanhar', 'acomapnahar', 'acomapnhar', 'acom

In [163]:
print(X_train_tfidf)

  (0, 2532)	0.15353764378298113
  (0, 2477)	0.131306459734973
  (0, 6327)	0.21007850228277614
  (0, 6065)	0.1707885559224741
  (0, 4632)	0.2621938973116595
  (0, 5024)	0.2007956494069003
  (0, 1849)	0.17837319898229415
  (0, 5702)	0.09455250065292875
  (0, 3250)	0.25729408019916467
  (0, 3646)	0.18765605185816997
  (0, 1793)	0.5058220888715674
  (0, 2504)	0.16692880207289376
  (0, 8742)	0.1819929039048207
  (0, 123)	0.18050701868048546
  (0, 7125)	0.21479301680101107
  (0, 4681)	0.2489461062984902
  (0, 543)	0.13504833908783312
  (0, 8526)	0.1550929787123076
  (0, 261)	0.1295546066314998
  (0, 406)	0.1478398144073415
  (0, 5352)	0.16014163449386984
  (0, 59)	0.18353383427596034
  (0, 8418)	0.13360540726290282
  (0, 4356)	0.13783116358136763
  (1, 1581)	0.6549356958477865
  :	:
  (1604, 3918)	0.09469636274548093
  (1604, 5318)	0.09334428803056992
  (1604, 2773)	0.09418193577605473
  (1604, 2112)	0.10780831926910271
  (1604, 5887)	0.10282790048475718
  (1604, 7800)	0.08992099888969073
  

In [164]:
#WORD2VEC

In [179]:

# Train a word2vec model on the tokenized text
sentences = [doc.split() for doc in train_set_filtered['text_lemmatized']]

model = Word2Vec(sentences, min_count=1)

# Get the word vectors
word_vectors = model.wv





In [180]:
# Calculate the weighted average vectors for each referral
referral_vectors = []
for i in range(X_train_tfidf.shape[0]):
    # Get the non-zero indices and values in the row
    row = X_train_tfidf.getrow(i)
    non_zero_indices = row.indices
    non_zero_values = row.data

    # Check if the row contains any non-zero values
    if len(non_zero_indices) > 0:
        # Get the word vectors for the non-zero indices
        word_vectors_for_referral = []
        for index, value in zip(non_zero_indices, non_zero_values):
            word = vocab[index]
            try:
                word_vector = word_vectors[word]
                word_vectors_for_referral.append(word_vector * value)
            except KeyError:
                pass

        # Calculate the average vector for the referral
        referral_vector = np.sum(word_vectors_for_referral, axis=0) / np.sum(non_zero_values)
        referral_vectors.append(referral_vector)
    else:
        # If the row contains all zeros, create a zero vector
        referral_vectors.append(np.zeros(model.vector_size))

# Assign the referral vectors to a new column in the dataframe
train_set['Referral Vectors'] = referral_vectors


ValueError: Length of values (1606) does not match length of index (1609)

In [167]:
# Calculate the weighted average vectors for each referral
referral_vectors = []
for i in range(X_train_tfidf.shape[0]):
    # Get the non-zero indices and values in the row
    row = X_train_tfidf.getrow(i)
    non_zero_indices = row.indices
    non_zero_values = row.data

    # Check if the row contains any non-zero values
    if len(non_zero_indices) > 0:
        # Get the word vectors for the non-zero indices
        word_vectors_for_referral = []
        for index, value in zip(non_zero_indices, non_zero_values):
            word = vocab[index]
            try:
                word_vector = word_vectors[word]
                word_vectors_for_referral.append(word_vector * value)
            except KeyError:
                pass

        # Calculate the average vector for the referral
        referral_vector = sum(word_vectors_for_referral) / sum(non_zero_values)
        referral_vectors.append(referral_vector)
    else:
        # If the row contains all zeros, create a zero vector
        referral_vectors.append(np.zeros(model.vector_size))

# Assign the referral vectors to a new column in the dataframe
X_train_tfidf['Referral Vectors'] = referral_vectors

IndexError: Index dimension must be 1 or 2

In [166]:
# Calculate the weighted average vectors for each referral
referral_vectors = []
for i in range(X_train_tfidf.shape[0]):
    # Get the non-zero indices and values in the row
    row = X_train_tfidf.getrow(i)
    non_zero_indices = row.indices
    non_zero_values = row.data

    # Check if the row contains any non-zero values
    if len(non_zero_indices) > 0:
        # Get the word vectors for the non-zero indices
        word_vectors_for_referral = []
        for index in non_zero_indices:
            word = vocab[index]
            try:
                word_vector = word_vectors[word]
                word_vectors_for_referral.append(word_vector * non_zero_values[index])
            except KeyError:
                pass

        # Calculate the average vector for the referral
        referral_vector = sum(word_vectors_for_referral) / len(word_vectors_for_referral)
        referral_vectors.append(referral_vector)
    else:
        # If the row contains all zeros, create a zero vector
        referral_vectors.append(np.zeros(model.vector_size))

# Assign the referral vectors to a new column in the dataframe
X_train_tfidf['Referral Vectors'] = referral_vectors


IndexError: index 2532 is out of bounds for axis 0 with size 24

In [156]:
# Calculate the weighted average vectors for each referral
referral_vectors = []
for row in X_train_tfidf:
    # Get the non-zero indices and values in the row
    non_zero_indices = row.indices
    print(non_zero_indices)
    non_zero_values = row.data
    print(non_zero_values)

    # Get the word vectors for the non-zero indices
    word_vectors_for_referral = []
    for index in non_zero_indices:
        word = vocab[index]
        try:
            word_vector = word_vectors[word]
            word_vectors_for_referral.append(word_vector * non_zero_values[index])
        except KeyError:
            pass

    # Calculate the average vector for the referral
    if word_vectors_for_referral:
        referral_vector = sum(word_vectors_for_referral) / len(word_vectors_for_referral)
        referral_vectors.append(referral_vector)
    else:
        referral_vectors.append(np.zeros(model.vector_size)) # If all values are zero, create a zero vector

# Assign the referral vectors to a new column in the dataframe
#['Referral Vectors'] = referral_vectors

[2532 2477 6327 6065 4632 5024 1849 5702 3250 3646 1793 2504 8742  123
 7125 4681  543 8526  261  406 5352   59 8418 4356]
[0.15353764 0.13130646 0.2100785  0.17078856 0.2621939  0.20079565
 0.1783732  0.0945525  0.25729408 0.18765605 0.50582209 0.1669288
 0.1819929  0.18050702 0.21479302 0.24894611 0.13504834 0.15509298
 0.12955461 0.14783981 0.16014163 0.18353383 0.13360541 0.13783116]


IndexError: index 2532 is out of bounds for axis 0 with size 24

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# Train a word2vec model
sentences = [text.split() for text in data['clean_text']]
model = Word2Vec(sentences, window=5, min_count=5, workers=4)



In [21]:
# Get the TF-IDF scores
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(data['clean_text'])




<1x10180 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [17]:
# Multiply each word embedding by its corresponding TF-IDF score
doc_vectors = []
for i, sentence in enumerate(sentences):
    weighted_embeddings = []
    for word in sentence:
        if word in model.wv.index_to_key:
            tfidf_score = X_tfidf[i, vectorizer.vocabulary_[word]]
            weighted_embeddings.append(model.wv[word] * tfidf_score)
    if weighted_embeddings:
        doc_vectors.append(np.average(weighted_embeddings, axis=0))

# Normalize the resulting vectors
doc_vectors = np.array(doc_vectors)
doc_vectors = doc_vectors / np.sum(X_tfidf, axis=1).reshape(-1, 1)

# Add the new column to the dataframe
data['weighted_word2vec'] = doc_vectors.tolist()


KeyError: '/'

In [14]:
model = Word2Vec(preprocessed_data, size=100, window=5, min_count=1)

# Step 3: Multiply words with their TF-IDF scores
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_scores = tfidf.fit_transform(text_data)

for i, text in enumerate(preprocessed_data):
    word_scores = []
    for word in text:
        tfidf_score = tfidf_scores[i, tfidf.vocabulary_[word]]
        word_vec = model.wv[word]
        word_scores.append(tfidf_score * word_vec)
    text_vec = sum(word_scores) / len(word_scores)
    print(text_data[i], text_vec)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


# define the Word2Vec model
model = Word2Vec(size=100, window=5, min_count=1)

# define the column transformer to select the "texto" column
ct = ColumnTransformer(
    [('texto', TfidfVectorizer(), 'texto')],
    remainder='passthrough'
)

# define the pipeline to apply the Word2Vec model
pipeline = Pipeline([
    ('column_transformer', ct),
    ('word2vec', model.wv)
])

# fit the pipeline to your dataset
pipeline.fit(your_data_frame)